In [137]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns; sns.set(style="ticks", color_codes=True)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [138]:
#all pitchers 1980-2022
sh = pd.read_csv('ss_sp_1980-2022.csv')
sh

,Rk,Player,WAR,GS,Season,Age,Team,Lg,W,L,...,FIP,WHIP,H9,HR9,BB9,SO9,SO/BB,WAR.1,WAA,Pos
0,1,Dwight Gooden,12.2,35,1985,20,NYM,NL,24,4,...,2.13,0.965,6.4,0.4,2.2,8.7,3.88,12.2,9.8,1
1,2,Roger Clemens,11.9,34,1997,34,TOR,AL,21,7,...,2.25,1.030,7.0,0.3,2.3,10.0,4.29,11.9,9.4,1
2,3,Pedro Martínez,11.7,29,2000,28,BOS,AL,18,6,...,2.17,0.737,5.3,0.7,1.3,11.8,8.88,11.7,9.7,NaN
3,4,Randy Johnson,10.7,35,2002,38,ARI,NL,24,5,...,2.66,1.031,6.8,0.9,2.5,11.6,4.70,10.7,8.7,1
4,5,Roger Clemens,10.4,31,1990,27,BOS,AL,21,6,...,2.18,1.082,7.6,0.3,2.1,8.2,3.87,10.4,8.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12331,12332,José Bautista,-2.8,6,1995,30,SFG,NL,3,8,...,6.23,1.450,10.7,2.1,2.3,4.0,1.73,-2.8,-3.5,1
12332,12333,Roy Halladay,-2.8,13,2000,23,TOR,AL,4,7,...,6.47,2.202,14.2,1.9,5.6,5.9,1.05,-2.8,-3.4,NaN
12333,12334,Terry Mulholland,-2.8,24,1995,32,SFG,NL,5,13,...,5.26,1.530,11.5,1.5,2.3,3.9,1.71,-2.8,-4.0,1/H
12334,12335,Dave Hamilton,-3.1,1,1980,32,OAK,AL,0,3,...,6.92,2.400,13.2,1.8,8.4,6.9,0.82,-3.1,-2.6,NaN


In [139]:
#drop WAR.1
sh = sh.drop(['WAR.1'], axis=1)

In [140]:
#drop GS.1
sh = sh.drop(['GS.1'], axis=1)

In [141]:
#check one
sh[['Rk','ERA+','SO/BB']][sh['Rk']==6385]


,Rk,ERA+,SO/BB
6384,6385,404.0,NaN


In [142]:
#drop POS since all are P
sh = sh.drop(['Pos'], axis=1)

In [143]:
#replace empties with 0.0
sh = sh.fillna(0.0)

In [144]:
#Fangraphs all pitchers 1980-2022
fg = pd.read_csv('FG_sp_1980-2022_2.csv')
fg

,FG_Season,FG_Name,FG_Team,FG_BABIP,FG_LOB%,FG_WAR,FG_RA9-WAR,FG_BIP-Wins,FG_LOB-Wins,FG_FDP-Wins,...,FG_WPA/LI,FG_Clutch,FG_SD,FG_MD,FG_ERA-,FG_FIP-,FG_K%,FG_BB%,FG_E-F,FG_playerid
0,1980,Bob Lacey,OAK,0.259,1.000,0.4,0.8,0.1,0.4,0.5,...,0.35,0.04,0,0,0,50,0.152,0.030,-1.97,1007232
1,1981,Terry Leach,NYM,0.214,0.667,0.2,0.2,0.1,-0.1,0.0,...,0.21,-0.08,0,0,0,31,0.222,0.000,-1.00,1007427
2,1981,Tim Leary,NYM,0.000,1.000,0.1,0.1,0.1,0.0,0.0,...,0.11,0.02,0,0,0,34,0.429,0.143,-1.10,1007438
3,1981,Charlie Puleo,NYM,0.292,0.900,0.2,0.4,0.0,0.2,0.2,...,0.13,0.07,0,0,0,66,0.182,0.091,-2.23,1010565
4,1981,Billy Smith,HOU,0.200,1.000,0.1,0.5,0.2,0.2,0.4,...,0.25,0.01,0,0,0,85,0.037,0.037,-2.74,1012098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12331,2018,Jonny Venters,TBR,1.000,0.000,0.0,-0.4,-0.1,-0.2,-0.3,...,-0.21,-0.05,0,0,3290,369,0.167,0.333,119.84,7175
12332,2018,Ryan Meisinger,BAL,0.800,0.000,0.0,-0.4,-0.2,-0.2,-0.3,...,-0.21,0.00,0,0,3160,274,0.000,0.167,122.84,18105
12333,2019,Michael Feliz,PIT,0.750,0.000,-0.1,-0.4,-0.1,-0.1,-0.2,...,-0.24,-0.02,0,0,3110,1169,0.000,0.167,83.79,11903
12334,2017,T.J. McFarland,ARI,0.833,0.000,-0.1,-0.4,-0.2,-0.2,-0.4,...,-0.30,0.01,0,0,4142,482,0.000,0.250,167.84,3237


In [145]:
#create PlayerTeamSeason in fg and sh df
sh['PlayerTeamSeason'] = sh['Player'].str.replace(" ", "_") + '_' + sh['Team'] + '_' + sh['Season'].map(str)
fg['PlayerTeamSeason'] = fg['FG_Name'].str.replace(" ", "_") + '_' + fg['FG_Team'] + '_' + fg['FG_Season'].map(str)


In [146]:
#cols for merger
fg_cols = fg.drop(['FG_Season', 'FG_Name', 'FG_Team'], axis=1)

In [147]:
#merge stathead data
df_comb = pd.merge(sh, fg_cols, how="inner", on=['PlayerTeamSeason'])

In [148]:
#group by playerid and then remove players with only 1 season
df_comb = df_comb.groupby("FG_playerid", group_keys=False).filter(lambda x: x.shape[0] > 1)


In [149]:
#function to add next season WAR to record
def next_season(player):
    player = player.sort_values("Season")
    player["Next_WAR"] = player["WAR"].shift(-1)
    return player

#add Next_WAR to record
df_comb_prev_next = df_comb.groupby("FG_playerid", group_keys=False).apply(next_season)


In [150]:
#function to add previous season WAR to record
def prev_season(player):
    player = player.sort_values("Season")
    player["Prev_WAR"] = player["WAR"].shift(1)
    return player

#Add Prev_WAR to record
df_comb_prev_next = df_comb_prev_next.groupby("FG_playerid", group_keys=False).apply(prev_season)

In [151]:
#check a player
df_comb_prev_next[["Player","Season","Prev_WAR","WAR","Next_WAR"]][df_comb_prev_next["Player"] == "Roger Clemens"]


,Player,Season,Prev_WAR,WAR,Next_WAR
2599,Roger Clemens,1984,NaN,1.8,2.7
1746,Roger Clemens,1985,1.8,2.7,8.8
17,Roger Clemens,1986,2.7,8.8,9.4
11,Roger Clemens,1987,8.8,9.4,7.1
102,Roger Clemens,1988,9.4,7.1,5.7
280,Roger Clemens,1989,7.1,5.7,10.4
3,Roger Clemens,1990,5.7,10.4,7.9
53,Roger Clemens,1991,10.4,7.9,8.7
20,Roger Clemens,1992,7.9,8.7,2.6
1846,Roger Clemens,1993,8.7,2.6,6.0


In [152]:
#create copy w/o Next and Prev WAR
df3 = df_comb

In [153]:
#check a player
df3[["Player","Season","Age","WAR"]][df3["Player"] == "Roger Clemens"]


,Player,Season,Age,WAR
1,Roger Clemens,1997,34,11.9
3,Roger Clemens,1990,27,10.4
11,Roger Clemens,1987,24,9.4
17,Roger Clemens,1986,23,8.8
20,Roger Clemens,1992,29,8.7
37,Roger Clemens,1998,35,8.1
53,Roger Clemens,1991,28,7.9
56,Roger Clemens,2005,42,7.8
64,Roger Clemens,1996,33,7.7
102,Roger Clemens,1988,25,7.1


In [154]:
#get only ID and Name
df_obj = df3[["FG_playerid","Player"]]
df_obj

,FG_playerid,Player
0,1004852,Dwight Gooden
1,815,Roger Clemens
2,60,Randy Johnson
3,815,Roger Clemens
4,1943,Zack Greinke
...,...,...
10338,1002255,Jim Clancy
10339,1011072,Jeff Robinson
10340,1303,Roy Halladay
10341,665,Terry Mulholland


In [155]:
#get only ID, Player, Pos
df_obj_pos = df3[["FG_playerid","Player","PlayerTeamSeason"]]
df_obj_pos

,FG_playerid,Player,PlayerTeamSeason
0,1004852,Dwight Gooden,Dwight_Gooden_NYM_1985
1,815,Roger Clemens,Roger_Clemens_TOR_1997
2,60,Randy Johnson,Randy_Johnson_ARI_2002
3,815,Roger Clemens,Roger_Clemens_BOS_1990
4,1943,Zack Greinke,Zack_Greinke_KCR_2009
...,...,...,...
10338,1002255,Jim Clancy,Jim_Clancy_HOU_1989
10339,1011072,Jeff Robinson,Jeff_Robinson_PIT_1989
10340,1303,Roy Halladay,Roy_Halladay_TOR_2000
10341,665,Terry Mulholland,Terry_Mulholland_SFG_1995


In [156]:
#get rid of duplicates
df_obj_filt = df_obj.drop_duplicates().sort_values(by=['FG_playerid'])
df_obj_filt

,FG_playerid,Player
13,3,Kevin Appier
6750,7,Mickey Callaway
3570,8,Dennis Cook
2069,20,Al Levine
3946,30,Lou Pote
...,...,...
3772,1014362,Anthony Young
1897,1014368,Curt Young
398,1014385,Matt Young
1411,1014407,Pat Zachry


In [157]:
#get peak P records
df_comb_prev_next_peak = df_comb_prev_next[(df_comb_prev_next['Age'] == 26)]
df_comb_prev_next_peak


,Rk,Player,WAR,GS,Season,Age,Team,Lg,W,L,...,FG_SD,FG_MD,FG_ERA-,FG_FIP-,FG_K%,FG_BB%,FG_E-F,FG_playerid,Next_WAR,Prev_WAR
580,675,Kevin Appier,4.5,23,1994,26,KCR,AL,7,6,...,0,0,79,75,0.222,0.096,0.35,3,4.6,9.3
5008,5854,Scott Schoeneweis,0.5,27,2000,26,ANA,AL,7,10,...,0,0,111,105,0.105,0.090,0.34,33,1.2,NaN
4015,4673,Aaron Sele,0.9,29,1996,26,BOS,AL,7,11,...,0,0,105,83,0.190,0.093,1.30,34,0.3,0.7
3441,4003,Scot Shields,1.3,1,2002,26,ANA,AL,5,3,...,0,0,135,83,0.143,0.143,2.37,35,3.1,NaN
916,1060,Jarrod Washburn,3.8,30,2001,26,ANA,AL,11,10,...,0,0,85,98,0.155,0.066,-0.60,40,4.5,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6053,7098,Rich Yett,0.2,12,1989,26,CLE,AL,5,6,...,0,0,136,117,0.095,0.103,0.92,1014342,NaN,1.2
9701,11496,Mike York,-0.9,4,1991,26,CLE,AL,1,4,...,0,0,233,118,0.140,0.105,4.79,1014353,NaN,0.2
8838,10407,Anthony Young,-0.5,13,1992,26,NYM,NL,2,14,...,0,0,138,89,0.141,0.050,1.71,1014362,-0.4,1.0
2312,2676,Curt Young,2.1,27,1986,26,OAK,AL,13,9,...,0,0,91,95,0.143,0.067,-0.28,1014368,2.5,-1.0


In [158]:
#Rename peak cols
df_comb_prev_next_peak = df_comb_prev_next_peak.rename(columns={"Rk":"Peak_Rk","WAR":"Peak_WAR","GS":"Peak_GS","Season":"Peak_Season","Age":"Peak_Age","Team":"Peak_Team","Lg":"Peak_Lg","W":"Peak_W","L":"Peak_L","W-L%":"Peak_W-L%","Dec":"Peak_Dec","ERA":"Peak_ERA","G":"Peak_G","CG":"Peak_CG","SHO":"Peak_SHO","SV":"Peak_SV","IP":"Peak_IP","H":"Peak_H","R":"Peak_R","ER":"Peak_ER","HR":"Peak_HR","BB":"Peak_BB","IBB":"Peak_IBB","SO":"Peak_SO","HBP":"Peak_HBP","BK":"Peak_BK","WP":"Peak_WP","BF":"Peak_BF","ERA+":"Peak_ERA+","FIP":"Peak_FIP","WHIP":"Peak_WHIP","H9":"Peak_H9","HR9":"Peak_HR9","BB9":"Peak_BB9","SO9":"Peak_SO9","SO/BB":"Peak_SO/BB","WAA":"Peak_WAA","FG_BABIP":"Peak_FG_BABIP","FG_LOB%":"Peak_FG_LOB%","FG_WAR":"Peak_FG_WAR","FG_RA9-WAR":"Peak_FG_RA9-WAR","FG_BIP-Wins":"Peak_FG_BIP-Wins","FG_LOB-Wins":"Peak_FG_LOB-Wins","FG_FDP-Wins":"Peak_FG_FDP-Wins","FG_K-BB%":"Peak_FG_K-BB%","FG_kwERA":"Peak_FG_kwERA","FG_TTO%":"Peak_FG_TTO%","FG_K/9+":"Peak_FG_K/9+","FG_BB/9+":"Peak_FG_BB/9+","FG_K/BB+":"Peak_FG_K/BB+","FG_H/9+":"Peak_FG_H/9+","FG_HR/9+":"Peak_FG_HR/9+","FG_AVG+":"Peak_FG_AVG+","FG_WHIP+":"Peak_FG_WHIP+","FG_BABIP+":"Peak_FG_BABIP+","FG_LOB%+":"Peak_FG_LOB%+","FG_K%+":"Peak_FG_K%+","FG_BB%+":"Peak_FG_BB%+","FG_Events":"Peak_FG_Events","FG_WPA":"Peak_FG_WPA","FG_-WPA":"Peak_FG_-WPA","FG_+WPA":"Peak_FG_+WPA","FG_RE24":"Peak_FG_RE24","FG_REW":"Peak_FG_REW","FG_pLI":"Peak_FG_pLI","FG_inLI":"Peak_FG_inLI","FG_gmLI":"Peak_FG_gmLI","FG_exLI":"Peak_FG_exLI","FG_Pulls":"Peak_FG_Pulls","FG_WPA/LI":"Peak_FG_WPA/LI","FG_Clutch":"Peak_FG_Clutch","FG_SD":"Peak_FG_SD","FG_MD":"Peak_FG_MD","FG_ERA-":"Peak_FG_ERA-","FG_FIP-":"Peak_FG_FIP-","FG_K%":"Peak_FG_K%","FG_BB%":"Peak_FG_BB%","FG_E-F":"Peak_FG_E-F","Next_WAR":"Peak_Next_WAR","Prev_WAR":"Peak_Prev_WAR", "PlayerTeamSeason":"Peak_PlayerTeamSeason"})
df_comb_prev_next_peak


,Peak_Rk,Player,Peak_WAR,Peak_GS,Peak_Season,Peak_Age,Peak_Team,Peak_Lg,Peak_W,Peak_L,...,Peak_FG_SD,Peak_FG_MD,Peak_FG_ERA-,Peak_FG_FIP-,Peak_FG_K%,Peak_FG_BB%,Peak_FG_E-F,FG_playerid,Peak_Next_WAR,Peak_Prev_WAR
580,675,Kevin Appier,4.5,23,1994,26,KCR,AL,7,6,...,0,0,79,75,0.222,0.096,0.35,3,4.6,9.3
5008,5854,Scott Schoeneweis,0.5,27,2000,26,ANA,AL,7,10,...,0,0,111,105,0.105,0.090,0.34,33,1.2,NaN
4015,4673,Aaron Sele,0.9,29,1996,26,BOS,AL,7,11,...,0,0,105,83,0.190,0.093,1.30,34,0.3,0.7
3441,4003,Scot Shields,1.3,1,2002,26,ANA,AL,5,3,...,0,0,135,83,0.143,0.143,2.37,35,3.1,NaN
916,1060,Jarrod Washburn,3.8,30,2001,26,ANA,AL,11,10,...,0,0,85,98,0.155,0.066,-0.60,40,4.5,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6053,7098,Rich Yett,0.2,12,1989,26,CLE,AL,5,6,...,0,0,136,117,0.095,0.103,0.92,1014342,NaN,1.2
9701,11496,Mike York,-0.9,4,1991,26,CLE,AL,1,4,...,0,0,233,118,0.140,0.105,4.79,1014353,NaN,0.2
8838,10407,Anthony Young,-0.5,13,1992,26,NYM,NL,2,14,...,0,0,138,89,0.141,0.050,1.71,1014362,-0.4,1.0
2312,2676,Curt Young,2.1,27,1986,26,OAK,AL,13,9,...,0,0,91,95,0.143,0.067,-0.28,1014368,2.5,-1.0


In [159]:
#merge ss and peak
ss_p_df = pd.merge(df_comb_prev_next, df_comb_prev_next_peak, how="inner", on=['FG_playerid'])

In [160]:
#Drop and Rename player cols
ss_p_df = ss_p_df.drop(['Player_y'], axis=1)
ss_p_df = ss_p_df.rename(columns={"Player_x":"Player"})

In [161]:
#Check
ss_p_df

,Rk,Player,WAR,GS,Season,Age,Team,Lg,W,L,...,Peak_FG_Clutch,Peak_FG_SD,Peak_FG_MD,Peak_FG_ERA-,Peak_FG_FIP-,Peak_FG_K%,Peak_FG_BB%,Peak_FG_E-F,Peak_Next_WAR,Peak_Prev_WAR
0,11156,Kevin Appier,-0.8,5,1989,21,KCR,AL,1,4,...,0.09,0,0,79,75,0.222,0.096,0.35,4.6,9.3
1,406,Kevin Appier,5.3,24,1990,22,KCR,AL,12,8,...,0.09,0,0,79,75,0.222,0.096,0.35,4.6,9.3
2,1450,Kevin Appier,3.3,31,1991,23,KCR,AL,13,10,...,0.09,0,0,79,75,0.222,0.096,0.35,4.6,9.3
3,53,Kevin Appier,8.0,30,1992,24,KCR,AL,15,8,...,0.09,0,0,79,75,0.222,0.096,0.35,4.6,9.3
4,16,Kevin Appier,9.3,34,1993,25,KCR,AL,18,8,...,0.09,0,0,79,75,0.222,0.096,0.35,4.6,9.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6550,12075,Matt Young,-1.4,4,1989,30,OAK,AL,1,4,...,-0.40,0,0,118,93,0.142,0.080,0.97,2.0,-0.9
6551,2736,Matt Young,2.0,33,1990,31,SEA,AL,8,18,...,-0.40,0,0,118,93,0.142,0.080,0.97,2.0,-0.9
6552,8942,Matt Young,-0.2,16,1991,32,BOS,AL,3,7,...,-0.40,0,0,118,93,0.142,0.080,0.97,2.0,-0.9
6553,7510,Matt Young,0.1,8,1992,33,BOS,AL,0,4,...,-0.40,0,0,118,93,0.142,0.080,0.97,2.0,-0.9


In [162]:
#exceed Prev_WAR in Peak? 0 if no, 1 if yes
#exceed P_WAR in Next_Peak?  0 if no, 1 if yes
def prev_war(x):
    if x["Peak_Prev_WAR"] >= x["Peak_WAR"]:
        return 0
    if x["Peak_Prev_WAR"] < x["Peak_WAR"]:
        return 1
    return ''

ss_p_df['Exceed_Prev_WAR']=ss_p_df.apply(lambda x: prev_war(x), axis=1)

def p_war(x):
    if x["Peak_WAR"] >= x["Peak_Next_WAR"]:
        return 0
    if x["Peak_WAR"] < x["Peak_Next_WAR"]:
        return 1
    return ''

ss_p_df['Exceed_Peak_WAR']=ss_p_df.apply(lambda x: p_war(x), axis=1)


In [163]:
#check one
ss_p_df[["FG_playerid","Season","Peak_Season","Age","Peak_Age","Player","WAR","Peak_Prev_WAR","Peak_WAR","Peak_Next_WAR","Exceed_Prev_WAR","Exceed_Peak_WAR"]][ss_p_df["Player"] == "Kevin Appier"]


,FG_playerid,Season,Peak_Season,Age,Peak_Age,Player,WAR,Peak_Prev_WAR,Peak_WAR,Peak_Next_WAR,Exceed_Prev_WAR,Exceed_Peak_WAR
0,3,1989,1994,21,26,Kevin Appier,-0.8,9.3,4.5,4.6,0,1
1,3,1990,1994,22,26,Kevin Appier,5.3,9.3,4.5,4.6,0,1
2,3,1991,1994,23,26,Kevin Appier,3.3,9.3,4.5,4.6,0,1
3,3,1992,1994,24,26,Kevin Appier,8.0,9.3,4.5,4.6,0,1
4,3,1993,1994,25,26,Kevin Appier,9.3,9.3,4.5,4.6,0,1
5,3,1994,1994,26,26,Kevin Appier,4.5,9.3,4.5,4.6,0,1
6,3,1995,1994,27,26,Kevin Appier,4.6,9.3,4.5,4.6,0,1
7,3,1996,1994,28,26,Kevin Appier,5.9,9.3,4.5,4.6,0,1
8,3,1997,1994,29,26,Kevin Appier,5.6,9.3,4.5,4.6,0,1
9,3,1998,1994,30,26,Kevin Appier,-0.2,9.3,4.5,4.6,0,1


In [164]:
#clean up objects

#drop player
ss_p_df = ss_p_df.drop(['PlayerTeamSeason','Peak_PlayerTeamSeason'], axis=1)

#AL = 0, NL = 1
ss_p_df['Lg'] = ss_p_df['Lg'].map({'AL':0, 'NL':1})
ss_p_df['Peak_Lg'] = ss_p_df['Peak_Lg'].map({'AL':0, 'NL':1})

#Team
ss_p_df['Team'] = ss_p_df['Team'].map({'ARI':0,'ATL':1,'BAL':2,'BOS':3,'CAL':4,'ANA':4,'LAA':4,'CHC':5,'CHW':6,'CIN':7,'CLE':8,'COL':9,'DET':10,'FLA':11,'MIA':11,'HOU':12,'KCR':13,'LAD':14,'MIL':15,'MIN':16,'NYM':17,'NYY':18,'OAK':19,'PHI':20,'PIT':21,'SDP':22,'SEA':23,'SFG':24,'STL':25,'TBD':26,'TBR':26,'TEX':27,'TOR':28,'WSN':29,'MON':29})
ss_p_df['Peak_Team'] = ss_p_df['Peak_Team'].map({'ARI':0,'ATL':1,'BAL':2,'BOS':3,'CAL':4,'ANA':4,'LAA':4,'CHC':5,'CHW':6,'CIN':7,'CLE':8,'COL':9,'DET':10,'FLA':11,'MIA':11,'HOU':12,'KCR':13,'LAD':14,'MIL':15,'MIN':16,'NYM':17,'NYY':18,'OAK':19,'PHI':20,'PIT':21,'SDP':22,'SEA':23,'SFG':24,'STL':25,'TBD':26,'TBR':26,'TEX':27,'TOR':28,'WSN':29,'MON':29})


In [165]:
#all teams 1980-2022
sh_td = pd.read_csv('team_data.csv')
sh_td.head(10)

,Season,Team,Team_GP,Team_WL
0,2002,LAA,162,0.611
1,2004,LAA,162,0.568
2,1998,LAA,162,0.525
3,1997,LAA,162,0.519
4,2000,LAA,162,0.506
5,2003,LAA,162,0.475
6,2001,LAA,162,0.463
7,1999,LAA,162,0.432
8,1999,ARI,162,0.617
9,2002,ARI,162,0.605


In [166]:
sh_td['Team'] = sh_td['Team'].map({'ARI':0,'ATL':1,'BAL':2,'BOS':3,'CAL':4,'ANA':4,'LAA':4,'CHC':5,'CHW':6,'CIN':7,'CLE':8,'COL':9,'DET':10,'FLA':11,'MIA':11,'HOU':12,'KCR':13,'LAD':14,'MIL':15,'MIN':16,'NYM':17,'NYY':18,'OAK':19,'PHI':20,'PIT':21,'SDP':22,'SEA':23,'SFG':24,'STL':25,'TBD':26,'TBR':26,'TEX':27,'TOR':28,'WSN':29,'MON':29})


In [167]:
ss_p_df['SeasonTeam'] = ss_p_df['Season'].map(str) + '_' + ss_p_df['Team'].map(str)
sh_td['SeasonTeam'] = sh_td['Season'].map(str) + '_' + sh_td['Team'].map(str)

In [168]:
ss_p_df['SeasonTeam'] = ss_p_df['Season'].map(str) + '_' + ss_p_df['Team'].map(str)

In [169]:
team_data = sh_td.copy()
team_data = team_data.drop(['Season','Team'], axis=1)
team_data


,Team_GP,Team_WL,SeasonTeam
0,162,0.611,2002_4
1,162,0.568,2004_4
2,162,0.525,1998_4
3,162,0.519,1997_4
4,162,0.506,2000_4
...,...,...,...
1223,162,0.401,2021_29
1224,162,0.364,2009_29
1225,161,0.366,2008_29
1226,162,0.340,2022_29


In [170]:
#merge team data
ss_p_df = pd.merge(ss_p_df, team_data, how="inner", on=['SeasonTeam'])
ss_p_df = ss_p_df.drop(['SeasonTeam'], axis=1)
ss_p_df["Games_played"] = ss_p_df["GS"]/ss_p_df["Team_GP"]
ss_p_df = ss_p_df.drop(['Team_GP'], axis=1)
ss_p_df

,Rk,Player,WAR,GS,Season,Age,Team,Lg,W,L,...,Peak_FG_FIP-,Peak_FG_K%,Peak_FG_BB%,Peak_FG_E-F,Peak_Next_WAR,Peak_Prev_WAR,Exceed_Prev_WAR,Exceed_Peak_WAR,Team_WL,Games_played
0,11156,Kevin Appier,-0.8,5,1989,21,13,0,1,4,...,75,0.222,0.096,0.35,4.6,9.3,0,1,0.568,0.030864
1,1400,Tom Gordon,3.3,16,1989,21,13,0,17,9,...,96,0.187,0.129,-0.16,1.4,3.6,0,0,0.568,0.098765
2,2419,Luis Aquino,2.3,16,1989,25,13,0,6,8,...,129,0.076,0.109,-2.81,3.1,2.3,0,1,0.568,0.098765
3,5333,Floyd Bannister,0.7,14,1989,34,13,0,4,1,...,94,0.163,0.075,0.72,4.7,4.2,0,1,0.568,0.086420
4,209,Mark Gubicza,6.3,36,1989,26,13,0,15,11,...,74,0.163,0.059,0.32,1.2,7.7,0,0,0.568,0.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6551,2417,Mike Witt,2.3,26,1982,21,4,0,8,6,...,92,0.180,0.079,-0.17,1.5,6.1,0,0,0.574,0.160494
6552,12323,Rick Sutcliffe,-2.5,10,1980,24,14,1,3,9,...,93,0.157,0.108,-1.01,3.1,-0.2,1,0,0.564,0.061350
6553,2122,Bob Welch,2.6,32,1980,23,14,1,14,9,...,87,0.188,0.087,-0.51,1.6,3.2,1,0,0.564,0.196319
6554,2954,Frank Tanana,1.9,31,1980,26,4,0,11,12,...,88,0.129,0.052,0.56,1.1,NaN,,0,0.406,0.193750


In [171]:
#export out
from pathlib import Path  
filepath = Path('ss_p_df_clean.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
ss_p_df.to_csv(filepath,index=False)  